In [30]:
import pandas as pd
import plotly.express as px

## Calificar examen estudiantes: Dimensión 7

In [49]:
est = pd.read_excel('../../../data/raw/Est_Con_Pre_110522.xlsx')
print(est.shape)
est.columns

(9500, 102)


Index(['Timestamp', 'Número de lista', 'Edad', 'Sexo', 'Sector vivienda',
       'Internet', 'Uso del dispositivo móvil', 'Nivel escolaridad madre',
       'Nivel escolaridad padre', 'Ocupación madre',
       ...
       'Tipo de discapacidad',
       '¿Te reconoces como una persona con algún tipo de discapacidad?',
       '1.8 Desarrollo de software', 'Conoce GreenTIC', 'Código IE', 'Grupo',
       'eliminar - ¿Qué opinas de cada una de estas carreras? [Fila 8]',
       'eliminar - ¿Qué opinas de cada una de estas carreras? [Fila 7]',
       'eliminar - ¿Qué opinas de cada una de estas carreras? [0,/]',
       'eliminar - ¿Qué opinas de cada una de estas carreras? [Fila 3]'],
      dtype='object', length=102)

In [50]:
est['Timestamp'] = pd.to_datetime(est['Timestamp'])
est = est[est.Timestamp>'2022-04-14']
est['Fecha'] = est.Timestamp.dt.strftime('%d/%m')
est['Fecha'][:5]

0    19/04
1    19/04
2    19/04
3    19/04
4    19/04
Name: Fecha, dtype: object

In [51]:
est = est.drop(columns=est.filter(regex=r'eliminar').columns)
est = est.drop(columns='Timestamp')
est = est.dropna(subset='Número de lista')
est = est[est['Código IE']<300]
est.shape

(9254, 71)

In [52]:
respuestas_correctas = {
    "Un algoritmo es:":"Una secuencia lógica de pasos para realizar una tarea",
    "¿Para qué sirven los algoritmos?":"Para planificar la solución a un problema",
    "Un bucle es:":"Un conjunto de instrucciones que se ejecuta mientras se cumpla una condición",
    "¿Cuál de las siguientes opciones sí le permite al robot completar la misión de fotografiar cada tortuga?":"a.",
    "¿Qué mensaje deseaba enviar la líder Wayuú?":"c. Nublado",
    "¿Cuál de los siguientes códigos permite que el robot complete su misión sembrando café?":"a.",
    "¿Cuál será la foto con más vistas?":"c) Julio",
    "Ayuda al robot verde a salir del laberinto":"b.",
    "Óscar lleva 2 loncheras a la escuela todos los días ¿Cuál de las siguientes afirmaciones es falsa?":"c) Si Óscar empaca Deditos para merendar, puede hacer Arroz de pollo para almorzar",
    "¿Cuál de las siguientes hamburguesas tiene los ingredientes A, E y F?":"a.",
    "¿Qué botella debe cambiarse de color para que el resultado final sea una botella de color blanco?":"a) La botella B debe ser verde"}

In [53]:
est.columns.tolist()

['Número de lista',
 'Edad',
 'Sexo',
 'Sector vivienda',
 'Internet',
 'Uso del dispositivo móvil',
 'Nivel escolaridad madre',
 'Nivel escolaridad padre',
 'Ocupación madre',
 'Ocupación padre',
 '¿Con quién vives?',
 'Grado',
 '1.1. Ingeniería',
 '2.1 Soy capaz de sacar buenas notas en esta asignatura',
 '2.2 Si me va bien en esta asignatura, me ayudará en mi futura ocupación',
 '2.3 A mis padres les gustaría que eligiera un futuro profesional relacionado a esta asignatura',
 '2.4 Sé de alguien en mi familia que utiliza conocimientos relacionados a esta asignatura en su ocupación',
 'comentarios 1-2',
 'Un algoritmo es:',
 '¿Para qué sirven los algoritmos?',
 'Un bucle es:',
 '3.1 Siento que soy capaz de explicar lo que es el pensamiento computacional',
 '3.2 Siento que puedo enumerar las sub-habilidades que componen el pensamiento computacional',
 '3.3 Siento que soy capaz de dar ejemplos para explicar las sub-habilidades del pensamiento computacional',
 '3.4 Siento que puedo expli

In [54]:
col_index = ['Edad', 'Sexo', 'Código IE', '¿Te reconoces como una persona con algún tipo de discapacidad?']
col_conocePC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[0]
col_PC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[1:5]
col_autoeficacia = est.filter(regex=r'^3.*', axis=1).columns.tolist()[-5:]
col_interes = est.filter(regex=r'^2.1|2.2', axis=1).columns.tolist()
col_conocimiento = list(respuestas_correctas.keys())
col_PC

['3.2 Siento que puedo enumerar las sub-habilidades que componen el pensamiento computacional',
 '3.3 Siento que soy capaz de dar ejemplos para explicar las sub-habilidades del pensamiento computacional',
 '3.4 Siento que puedo explicar la forma en que las sub-habilidades del pensamiento computacional se correlacionan con la programación',
 '3.5 Siento que puedo analizar un ejercicio y determinar qué sub-habilidades de pensamiento computacional busca desarrollar']

In [55]:
est = est.reset_index()

In [56]:
df_interes = est[['index']+col_interes].melt(id_vars='index', value_name='Seleccion', var_name='Pregunta')
df_interes['Interes en tecnologia'] = 1*(df_interes['Seleccion'].str.contains('Tecnología'))
df_puntaje_interes = df_interes.pivot_table(index='index', values=['Interes en tecnologia'], aggfunc=['sum', 'max']).reset_index()
df_puntaje_interes.columns = ['index', 'Puntaje interes', 'Interesado en tecnologia']
df_puntaje_interes['Puntaje interes'] = 100*(df_puntaje_interes['Puntaje interes']/2)
#df_puntaje_interes = (100*df_interes.groupby('index')['Interes en tecnologia'].sum()/4).reset_index()
df_puntaje_interes

,index,Puntaje interes,Interesado en tecnologia
0,1,100.0,1
1,2,50.0,1
2,3,50.0,1
3,4,100.0,1
4,5,0.0,0
...,...,...,...
9249,9495,50.0,1
9250,9496,0.0,0
9251,9497,100.0,1
9252,9498,0.0,0


In [57]:

df_conocimientos = est[['index']+col_conocimiento].melt(id_vars='index', value_name='Respuesta estudiante', var_name='Pregunta')
df_conocimientos['Respuesta correcta'] = df_conocimientos['Pregunta'].replace(respuestas_correctas)
df_conocimientos['Puntaje conocimiento'] = 1*(df_conocimientos['Respuesta estudiante'] == df_conocimientos['Respuesta correcta'])
df_puntaje = (df_conocimientos.groupby('index')['Puntaje conocimiento'].sum()*100/11).reset_index()
df_puntaje.describe()

,index,Puntaje conocimiento
count,9254.000000,9254.000000
mean,4764.779123,32.538264
std,2745.684477,19.742658
min,1.000000,0.000000
25%,2379.250000,18.181818
50%,4757.500000,27.272727
75%,7144.750000,45.454545
max,9499.000000,100.000000


In [58]:
df_7 = pd.merge(est, df_puntaje, on='index')
df_7 = pd.merge(df_7, df_puntaje_interes, on='index')
df_7

,index,Número de lista,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,...,Tipo de discapacidad,¿Te reconoces como una persona con algún tipo de discapacidad?,1.8 Desarrollo de software,Conoce GreenTIC,Código IE,Grupo,Fecha,Puntaje conocimiento,Puntaje interes,Interesado en tecnologia
0,1,3.0,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,...,NaN,No,Me interesa poco,No,166.0,B o 02,19/04,45.454545,100.0,1
1,2,7.0,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,...,NaN,No,Está entre mis preferidas,No,166.0,B o 02,19/04,18.181818,50.0,1
2,3,30.0,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,...,NaN,No,Me interesa poco,No,166.0,B o 02,19/04,36.363636,50.0,1
3,4,6.0,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,...,NaN,No,Está entre mis preferidas,No,166.0,B o 02,19/04,18.181818,100.0,1
4,5,10.0,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,...,NaN,No,La evitaría,No,166.0,B o 02,19/04,36.363636,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9249,9495,19.0,12.0,Masculino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Cuida el hogar,...,NaN,No,La evitaría,No,246.0,D o 04,11/05,36.363636,50.0,1
9250,9496,13.0,16.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Cuida el hogar,...,NaN,No,La evitaría,No,55.0,A o 01,11/05,27.272727,0.0,0
9251,9497,8.0,14.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,Titulo universitario,Trabaja,...,NaN,Sí,No la conozco,No,103.0,G o 07,11/05,72.727273,100.0,1
9252,9498,12.0,14.0,Femenino,Rural,Sí,Uso propio (solo lo usas tú),Primaria,Secundaría,Trabaja,...,NaN,No,No la conozco,No,189.0,A o 01,11/05,9.090909,0.0,0


In [59]:
df_7['Media nacional resultados'] = df_7['Puntaje conocimiento'].mean()
df_7['Conoce PC'] = 1*(df_7[col_conocePC] > 2)
df_7['Puntaje conocimientos PC'] = 100*(df_7[col_PC].mean(axis=1)/5)
df_7['Puntaje autoeficacia'] = (df_7[col_autoeficacia].mean(axis=1)*100/5)
df_7['Media nacional autoeficacia'] = df_7['Puntaje autoeficacia'].mean()


In [16]:
df_7[col_conocePC].value_counts()

3.0    2317
1.0    1663
2.0    1441
4.0    1419
5.0    1106
Name: 3.1 Siento que soy capaz de explicar lo que es el pensamiento computacional, dtype: int64

In [21]:
df_7['Media nacional interés'] = df_7['Puntaje interes'].mean()


In [22]:
df_7.to_excel('Estudiantes_10052022.xlsx', index=False)

In [23]:
cols_dimension7 = ['Puntaje conocimiento', 'Media nacional resultados','Puntaje interes', 'Interesado en tecnologia', 'Conoce PC', 'Puntaje conocimientos PC', 'Puntaje autoeficacia', 'Media nacional autoeficacia', 'Media nacional interés', 'index']
agrupadores_dimension7 = ['mean', 'max', 'mean', 'sum', 'sum', 'mean','mean', 'max', 'max', 'count']
agg_funcs = dict(zip(cols_dimension7, agrupadores_dimension7))

In [24]:
df_institucion = df_7.pivot_table(index=['Código IE'], values=cols_dimension7, aggfunc=agg_funcs).rename(columns={'index':'Cantidad estudiantes'})
df_institucion['% Conoce PC'] = (df_institucion['Conoce PC']/df_institucion['Cantidad estudiantes'])*100
df_institucion['% Interesado'] = (df_institucion['Interesado en tecnologia']/df_institucion['Cantidad estudiantes'])*100

In [25]:
df_institucion['Diferencia resultados'] = (df_institucion['Puntaje conocimiento'] - df_institucion['Media nacional resultados'])*100/df_institucion['Media nacional resultados']

df_institucion['Diferencia Autoeficacia'] = (df_institucion['Puntaje autoeficacia'] - df_institucion['Media nacional autoeficacia'])*100/df_institucion['Media nacional autoeficacia']

df_institucion['Diferencia interes'] = (df_institucion['Puntaje interes'] - df_institucion['Media nacional interés'])*100/df_institucion['Media nacional interés']

In [26]:
df_institucion

,Conoce PC,Interesado en tecnologia,Media nacional autoeficacia,Media nacional interés,Media nacional resultados,Puntaje autoeficacia,Puntaje conocimiento,Puntaje conocimientos PC,Puntaje interes,Cantidad estudiantes,% Conoce PC,% Interesado,Diferencia resultados,Diferencia Autoeficacia,Diferencia interes
Código IE,,,,,,,,,,,,,,,
1.0,90,109,66.752077,40.633652,32.216324,69.466667,28.363636,58.566667,36.333333,150,60.000000,72.666667,-11.958805,4.066675,-10.583146
2.0,35,53,66.752077,40.633652,32.216324,59.104478,24.830393,51.492537,47.388060,67,52.238806,79.104478,-22.926049,-11.456721,16.622694
3.0,41,43,66.752077,40.633652,32.216324,69.904762,29.870130,59.761905,39.285714,63,65.079365,68.253968,-7.282625,4.722977,-3.317294
4.0,24,35,66.752077,40.633652,32.216324,66.222222,33.737374,59.555556,46.666667,45,53.333333,77.777778,4.721364,-0.793765,14.847335
5.0,69,99,66.752077,40.633652,32.216324,73.981651,30.942452,61.422018,55.504587,109,63.302752,90.825688,-3.954119,10.830487,36.597584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250.0,37,50,66.752077,40.633652,32.216324,62.024096,23.767798,54.337349,24.397590,83,44.578313,60.240964,-26.224362,-7.082896,-39.957181
251.0,81,77,66.752077,40.633652,32.216324,66.909091,25.770098,59.504132,27.479339,121,66.942149,63.636364,-20.009192,0.235220,-32.372953
252.0,37,42,66.752077,40.633652,32.216324,62.714286,24.188312,57.946429,33.035714,56,66.071429,75.000000,-24.919082,-6.048937,-18.698634


In [27]:
df_institucion['Nivel ConocePC'] =  pd.cut(df_institucion['% Conoce PC'], bins=[0, 20, 40, 50, 60, 100], labels=['1A', '1B', '2A', '2B', '3'], include_lowest=True)

df_institucion['Nivel PuntajePC'] =  pd.cut(df_institucion['Puntaje conocimientos PC'], bins=[0, 50, 60,70, 100], labels=['1A', '1B',"2A",'3'], include_lowest=True)

df_institucion['Nivel Resultados'] = pd.cut(df_institucion['Diferencia resultados'], bins=[-100, -50, -30,  0, 10,20, 100], labels=['1A', '1B', "2A", "2B", '3', '4'], include_lowest=True)

df_institucion['Nivel Interes'] = pd.cut(df_institucion['% Interesado'], bins=[0, 20, 40, 60, 80, 100], labels=['1A', '1B', "2A", "2B", '4'], include_lowest=True)

df_institucion['Nivel Interes pares'] = pd.cut(df_institucion['Diferencia interes'], bins=[-100,-50, -30, 0 , 200], labels=['1B', "2A", "2B", '4'], include_lowest=True)

df_institucion['Nivel Autoeficacia'] = pd.cut(df_institucion['Puntaje autoeficacia'], bins=[0, 30, 60, 100], labels=['1B', "2A", '4'], include_lowest=True)

df_institucion['Nivel Autoeficacia pares'] = pd.cut(df_institucion['Diferencia Autoeficacia'], bins=[-100, 0, 10, 20, 100], labels=['2A',"2B", "3", '4'], include_lowest=True)


df_institucion

,Conoce PC,Interesado en tecnologia,Media nacional autoeficacia,Media nacional interés,Media nacional resultados,Puntaje autoeficacia,Puntaje conocimiento,Puntaje conocimientos PC,Puntaje interes,Cantidad estudiantes,...,Diferencia resultados,Diferencia Autoeficacia,Diferencia interes,Nivel ConocePC,Nivel PuntajePC,Nivel Resultados,Nivel Interes,Nivel Interes pares,Nivel Autoeficacia,Nivel Autoeficacia pares
Código IE,,,,,,,,,,,,,,,,,,,,,
1.0,90,109,66.752077,40.633652,32.216324,69.466667,28.363636,58.566667,36.333333,150,...,-11.958805,4.066675,-10.583146,2B,1B,2A,2B,2B,4,2B
2.0,35,53,66.752077,40.633652,32.216324,59.104478,24.830393,51.492537,47.388060,67,...,-22.926049,-11.456721,16.622694,2B,1B,2A,2B,4,2A,2A
3.0,41,43,66.752077,40.633652,32.216324,69.904762,29.870130,59.761905,39.285714,63,...,-7.282625,4.722977,-3.317294,3,1B,2A,2B,2B,4,2B
4.0,24,35,66.752077,40.633652,32.216324,66.222222,33.737374,59.555556,46.666667,45,...,4.721364,-0.793765,14.847335,2B,1B,2B,2B,4,4,2A
5.0,69,99,66.752077,40.633652,32.216324,73.981651,30.942452,61.422018,55.504587,109,...,-3.954119,10.830487,36.597584,3,2A,2A,4,4,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250.0,37,50,66.752077,40.633652,32.216324,62.024096,23.767798,54.337349,24.397590,83,...,-26.224362,-7.082896,-39.957181,2A,1B,2A,2B,2A,4,2A
251.0,81,77,66.752077,40.633652,32.216324,66.909091,25.770098,59.504132,27.479339,121,...,-20.009192,0.235220,-32.372953,3,1B,2A,2B,2A,4,2B
252.0,37,42,66.752077,40.633652,32.216324,62.714286,24.188312,57.946429,33.035714,56,...,-24.919082,-6.048937,-18.698634,3,1B,2A,2B,2B,4,2A


In [29]:
df_institucion['Nivel ConocePC'].value_counts()

3     78
2B    37
2A    11
1A     6
1B     3
Name: Nivel ConocePC, dtype: int64

In [28]:
df_institucion['Dimensión 7'] = df_institucion.filter(regex='^Nivel').min(axis=1)

In [30]:
df_institucion.to_excel('Dimensión 7.xlsx')

In [29]:
df_institucion['Dimensión 7']

Código IE
1.0      1B
2.0      1B
3.0      1B
4.0      1B
5.0      2A
         ..
250.0    1B
251.0    1B
252.0    1B
259.0    1A
297.0    2A
Name: Dimensión 7, Length: 135, dtype: object